In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input
from tensorflow.keras.layers import Flatten, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import math

In [4]:
#parameter

batch_size = 32 #실제 논문에서는 128
epochs = 200
data_augmentation = True
num_class = 10
n = 3

In [5]:
subtract_pixel_mean = True

In [6]:
version = 1
#ResNet v1 = 1
#ResNet v2 = 2

if version ==1:
    depth = n*6+2
elif version == 2:
    depth = n*9+2

In [7]:
model_type = 'ResNet%dv%d' % (depth,version)

In [10]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [11]:
input_shape = x_train.shape[1:]

In [15]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [16]:
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train,axis = 0)
    x_train -=x_train_mean
    x_test -= x_train_mean

In [17]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [19]:
y_train = to_categorical(y_train, num_class)
y_test = to_categorical(y_test, num_class)

In [20]:
def lr_schedule(epoch):
    lr = 1e-3
    
    if epoch > 180:
        lr *=0.5e-3
    elif epoch > 160:
        lr *=1e-3
    elif epoch >120:
        lr *=1e-2
    elif epoch >80:
        lr *= 1e-1
        
    print(lr)
    return lr

In [32]:
def layer_RESNET(inputs,
                num_filter = 16,
                kernel_size = 3,
                strides = 1,
                activation = 'relu',
                batch_normalization = True,
                conv_first = True):
    
    conv = Conv2D(num_filter,
                 kernel_size = kernel_size,
                 strides = strides,
                 padding = "same",
                 kernel_initializer = 'he_normal',
                 kernel_regularizer = l2(1e-4))
    
    x = inputs
    
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation != None:
            x = Activation(activation)(x)
        
    else :
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation != None:
            x = Activation(activation)(x)
        
        x = conv(x)
        
    return x

In [66]:
def resnetv1(input_shape , depth, num_class = 10,layer_num=3):
    
    if (depth-2)%6 !=0 :
        raise ValueError('depth should be 6n+2 current Depth : '+depth)
        
    num_fitler = 16
    num_res_block = int((depth-2)/6)
    
    inputs = Input(shape = input_shape)
    
    x = layer_RESNET(inputs = inputs)
    
    for stack in range(layer_num):
        
        for res_block in range(num_res_block):
            
            stride = 1
            
            if stack >0 and res_block == 0:
                stride = 2
            
            y = layer_RESNET(
                inputs = x,
                num_filter = num_fitler,
                strides = stride
            )
            y = layer_RESNET(
                inputs = y,
                num_filter = num_fitler,
                activation = None
            )
            
            if stack>0 and res_block ==0 :
                x = layer_RESNET(
                    inputs = x,
                    num_filter= num_fitler,
                    kernel_size = 1,
                    strides= stride,
                    activation = None,
                    batch_normalization = False
                )
                
            x = add([x,y])
            x = Activation('relu')(x)
        num_fitler *=2
        
    x = AveragePooling2D(pool_size = 8 )(x)
    y = Flatten()(x)
    
    outputs = Dense(
        num_class,
        activation = 'softmax',
        kernel_initializer = 'he_normal'
                   )(y)
    
    model = Model(inputs = inputs,outputs = outputs)
    return model

In [67]:
def resnet_v2(input_shape,depth,num_class = 10):
    
    if (depth-2)%9 !=0:
        raise ValueError('depth should be 9n+2 current Depth : '+depth)
        
    num_filters_in = 16
    num_res_blocks = int((depth-2)/9)
    
    inputs = Input(shape=input_shape)
    
    x = layer_RESNET(
        inputs = inputs,
        num_filters = num_filters_in,
        conv_first = True
    )
    
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides =1
            
            if stage ==0:
                num_filters_out = num_filters_in*4
                
                if res_block == 0:
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filter_in*2
                if res_block == 0:
                    strides = 2
                    
            y= layer_RESNET(
                inputs= x,
                num_filter=num_filters_in,
                kernel_size =1,
                strides=strides,
                activation= activation,
                batch_normalization=batch_normalization,
                conv_first=False
            )
            
            y = layer_RESNET(
                inputs=y,
                num_filter=num_filter_in,
                conv_first=False
            )
            
            y = layer_RESNET(
                inputs=y,
                num_filter=num_filters_out,
                conv_first=False
            )
            
            if res_block == 0:
                
                x = layer_RESNET(
                    inputs = x,
                    num_filter=num_filters_out,
                    kernel_size = 1,
                    strides=strides,
                    activation=None,
                    batch_normalization=False
                )
            x =add([x,y])
        num_filters_in =num_filters_out
        
        
        
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size = 8)(x)
    y = Flatten()(x)
    
    outputs = Dense(
        num_classes,
        activation = 'softmax',
        kernel_initializer = 'he_normal'
    )(y)
    
    model = Model(inputs = inputs,outputs = outputs)
    
    return model

In [68]:
##

if version == 2:
    model = resnet_v2(
        input_shape=input_shape,
        depth = depth
    )
    
else:
    model = resnetv1(
        input_shape = input_shape,
        depth = depth
    )
    
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = Adam(lr = lr_schedule(0)),
    metrics = ['acc']
)
model.summary()

0.001
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_129 (Conv2D)             (None, 32, 32, 16)   448         input_15[0][0]                   
__________________________________________________________________________________________________
batch_normalization_121 (BatchN (None, 32, 32, 16)   64          conv2d_129[0][0]                 
__________________________________________________________________________________________________
activation_117 (Activation)     (None, 32, 32, 16)   0           batch_normalization_121[0][0]    
______________________________________________________________________________________

In [69]:
print(model_type)

ResNet20v1


In [70]:
save_dir=os.path.join(os.getcwd(),'saved_models')

In [72]:
model_name = 'cifar10_%s_model.{epoch:03d}.h5' %model_type

In [73]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir,model_name)

In [74]:
checkpoint = ModelCheckpoint(
    filepath = filepath,
    monitor = 'val_acc',
    verbose = 1,
    save_best_only = True
)

In [75]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [76]:
lr_reducer = ReduceLROnPlateau(
    factor = np.sqrt(0.1),
    cooldown = 0,
    patience = 5,
    min_lr = 0.5e-6
)

In [77]:
callbacks = [checkpoint,lr_reducer,lr_scheduler]

In [85]:
if not data_augmentation:
    model.fit(
    x_train,
    y_train,
    epochs = epochs,
    validation_data=(x_test,y_test),
    shuffle = True,
    callbacks=callbacks)
    
else:
    
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False
     )
        
    datagen.fit(x_train)
    steps_per_epoch =  math.ceil(len(x_train) / batch_size)
    model.fit(
        x=datagen.flow(x_train, y_train, batch_size=batch_size),
        verbose=1,
        epochs=epochs,
        validation_data=(x_test, y_test),
        steps_per_epoch=steps_per_epoch,
        callbacks=callbacks
    )

Epoch 1/200
0.001
1563/1563 [==============================] - 437s 278ms/step - loss: 1.8207 - acc: 0.4056 - val_loss: 1.6834 - val_acc: 0.5022

Epoch 00001: val_acc improved from -inf to 0.50220, saving model to C:\Users\KTDS\Desktop\im\Resnet20\saved_models\cifar10_ResNet20v1_model.001.h5
Epoch 2/200
0.001
1563/1563 [==============================] - 426s 273ms/step - loss: 1.2230 - acc: 0.6249 - val_loss: 1.2628 - val_acc: 0.6233

Epoch 00002: val_acc improved from 0.50220 to 0.62330, saving model to C:\Users\KTDS\Desktop\im\Resnet20\saved_models\cifar10_ResNet20v1_model.002.h5
Epoch 3/200
0.001
1335/1563 [========================>.....] - ETA: 1:00 - loss: 1.0405 - acc: 0.6933

KeyboardInterrupt: 

In [86]:
scroes = model.evaluate(
    x_test,
    y_test,
    batch_size = batch_size,
    verbose = 0
)

[0.9501985311508179, 0.7296000123023987]